## 小象学院课程
## Pyhton数据分析
## 美国化学学会杂志-2
“美国化学学会杂志”是美国化学学会出版的首要期刊，也是所有化学领域排名最高的期刊之一。本项目的数据集是该期刊从1996到2016的论文集,用以分析作者发表论文的情况。

> **提示**：<br>
1、这样的文字将会指导你如何使用 jupyter Notebook 来完成项目。<br>
2、你可以通过单击代码区域，然后使用键盘快捷键 Shift+Enter 或 Shift+Return 来运行代码。或者在选择代码后使用执行（run cell）按钮执行代码。Markdown的文字区域也同样可以如此操作。<br>
3、在如下有# TODO 提示的地方，将代码补全，实现注释中所要求的功能。<br>
4、在有"** 回答：**" 提示的地方，回答其上所提出的问题。<br>


### 项目数据集介绍：
数据库对于数据分析是非常重要的，它可以存储大量有某种关系的表，通过对表进行连接操作，以实现对数据的潜在分析。
本项目采用的数据集以sqlite的方式进行存储，数据集中包括三个表：
* Authors：对作者进行描述
* Paper_Authors：对作者发表论文的情况进行描述
* Papers：关于论文的表

## 分析流程
这个作业的数据是基于上个作业存入的数据，作业偏向于对数据的处理了，让我们开始进阶吧。<br>
1 导入数据：获得作者的ID号列表和对应的论文数量<br>
2 分析论文的浏览次数：依据得到的作者的ID号列表，分析每个作者的论文被浏览次数，比如，浏览的总次数、浏览的平均次数、浏览的最多次数、浏览的最少次数<br>
3 分析发表论文的日期：依据作者的ID号列表，计算作者每年发表的论文次数，得到论文次数最多的年份，当论文次数最多的年份存在多个时，则保留最近的年份，同时，保留该年份对应的论文数量<br>
4 合并数据，存入文件：将作者ID号列表、论文数量、浏览次数、论文发表日期进行合并，将数据存入txt文件<br>

### 导入数据

导入必备的Python库

In [48]:
import sqlite3
import json
import datetime
import numpy as np
import pandas as pd

在上个作业中，我们已经得到了关于每个作者的ID号、名字和发表的论文数量的数据，现在我们导入上个作业保存的json数据，得到该作业的ID号列表和作者对应的论文数量。

In [49]:
# author_paper_data的每个元素列表，比如[5, 30]中的第一个元素代表了作者authorID号，第二个元素表示该作者发表的论文数量
author_paper_data = []
file_path = './dataFile/author_paper_data.json'
with open(file_path,'r') as f_obj:
    # TODO
    # 使用json模块导入上个作业保存的数据，得到的数据的ID号和发表的论文数量，将最终结果存储在author_paper_data列表中，
    load_dict=json.load(f_obj)
print(load_dict)
for item in load_dict:
    list1=[]
    list1.append(item['author_id'])
    list1.append(item['papers_num'])
    author_paper_data.append(list1)
authors_id,papers_num = zip(*author_paper_data)
print(author_paper_data)

[{'author_name': 'James Snyder', 'author_id': 5, 'papers_num': 30}, {'author_name': 'Kenneth Karlin', 'author_id': 18, 'papers_num': 75}, {'author_name': 'Per-Ola Norrby', 'author_id': 20, 'papers_num': 27}, {'author_name': 'K. Sharpless', 'author_id': 22, 'papers_num': 23}, {'author_name': 'M. Ghadiri', 'author_id': 26, 'papers_num': 27}, {'author_name': 'Kurt Gothelf', 'author_id': 29, 'papers_num': 21}, {'author_name': 'Karl Jørgensen', 'author_id': 31, 'papers_num': 43}, {'author_name': 'Ivano Bertini', 'author_id': 48, 'papers_num': 46}, {'author_name': 'Claudio Luchinat', 'author_id': 53, 'papers_num': 40}, {'author_name': 'Michael Wasielewski', 'author_id': 57, 'papers_num': 121}, {'author_name': 'Robert Grubbs', 'author_id': 64, 'papers_num': 93}, {'author_name': 'Joseph Ziller', 'author_id': 65, 'papers_num': 58}, {'author_name': 'Jay Winkler', 'author_id': 69, 'papers_num': 39}, {'author_name': 'Harry Gray', 'author_id': 70, 'papers_num': 72}, {'author_name': 'Stephen Lippard

连接数据库，得到游标

In [50]:
def connect_sqlite(sqlite_path):
    conn = sqlite3.connect(sqlite_path)
    conn.row_factory = sqlite3.Row 
    cur = conn.cursor()
    return conn, cur

conn, cur = connect_sqlite("./dataFile/database.sqlite")

### 得到每个作者的论文信息
* 根据authorID，将Papers和Paper_Authors表中的paperID进行内连接，获得每个authorID在papers表对应paper信息，在后面需要根据得到的paper信息进行论文浏览次数和论文发表日期的分析。

In [51]:
def get_papers_author_info(author_paper_data):
    """
    功能：
        根据author_paper_data获得papers数据
    参数：
        author_paper_data包含作者的authorID号和论文数量
    返回值：
        将authorID作为键，authorID对应的papers数据作为值，以字典的形式返回数据
    """   
    author_papers_dict = {}
    # 根据get_paper_author_info方法获得authorID对应的paper数据，将authorID作为键，authorID对应的papers数据作为值，以键值对的形式存储数据
    n = len(author_paper_data)
    for i in range(n):
        list2=[]
        author_id = author_paper_data[i][0]
        paper_num = author_paper_data[i][1]
        paper_author_info = cur.execute("select * from Papers as paper inner join Paper_Authors as paper_author on paper.paperID==paper_author.paperID and paper_author.authorID='%d'"%author_id)
        author_papers_dict[author_id] = paper_author_info.fetchall()
        # TODO
        # 依据每个作者的author_id，判断得到的author_papers_dict[author_id]的论文数量和文件导入的论文数量是否相等，
        # 若是相等，则打印作者的ID号列表和论文数量,eg."authorID号是：5,论文数量paper_num是：30"，否则结束循环
        for i in author_papers_dict[author_id]: 
            list2.append(i['title'])
        if len(list2)==paper_num:
            print("authorID号是:",author_id,"论文数量paper_num是:",paper_num)
        else:
            break 
    print(author_papers_dict)
    return author_papers_dict
# author_paper_data包含了作者的ID号和论文数量
author_papers_dict=get_papers_author_info(author_paper_data)

authorID号是: 5 论文数量paper_num是: 30
authorID号是: 18 论文数量paper_num是: 75
authorID号是: 20 论文数量paper_num是: 27
authorID号是: 22 论文数量paper_num是: 23
authorID号是: 26 论文数量paper_num是: 27
authorID号是: 29 论文数量paper_num是: 21
authorID号是: 31 论文数量paper_num是: 43
authorID号是: 48 论文数量paper_num是: 46
authorID号是: 53 论文数量paper_num是: 40
authorID号是: 57 论文数量paper_num是: 121
authorID号是: 64 论文数量paper_num是: 93
authorID号是: 65 论文数量paper_num是: 58
authorID号是: 69 论文数量paper_num是: 39
authorID号是: 70 论文数量paper_num是: 72
authorID号是: 73 论文数量paper_num是: 137
authorID号是: 74 论文数量paper_num是: 117
authorID号是: 79 论文数量paper_num是: 79
authorID号是: 89 论文数量paper_num是: 25
authorID号是: 91 论文数量paper_num是: 45
authorID号是: 92 论文数量paper_num是: 40
{5: [<sqlite3.Row object at 0x00000238C565DAF0>, <sqlite3.Row object at 0x00000238C5604070>, <sqlite3.Row object at 0x00000238C5604170>, <sqlite3.Row object at 0x00000238C5604230>, <sqlite3.Row object at 0x00000238C56042F0>, <sqlite3.Row object at 0x00000238C5604F90>, <sqlite3.Row object at 0x00000238C56043F0>, <sqli

### 对作者发表论文的浏览次数进行统计分析
每个作者会发表多个论文，每个论文被浏览的次数一般不同，现在对浏览次数进行求和、均值、最大值、最小值操作，以对每个authorID的论文的浏览次数进行统计分析。

In [52]:
def process_view_num(authors_id,author_papers_dict):
    """
    功能：
        根据authorID对每个作者的浏览次数进行求和，均值，最大值，最小值操作
    参数：
        authors_id：作者列表数据，包含作者的authorID号
        author_papers_dict：包含了每个作者的paper数据，键为authorID，值为paper数据
    返回值：
        sum_mean_max_min_view是一个二维数据结构，行数据是对浏览次数依次按照求和、均值、最大值、最小值的结果
    """   
    sum_mean_max_min_view = []
    viewslist=[]
    # TODO
    # 对authors_id进行遍历，根据authorID获得字典的值，即paper信息，然后使用"views"字段获得浏览次数，进而进行统计分析
    for i in authors_id:
        items=author_papers_dict[i]
        list0=[]
        for item in items:
            list0.append(item['views'])
        sum_views=sum(list0)
        mean_views=sum(list0)/len(list0)
        max_views=max(list0)
        min_views=min(list0)
        l0=[sum_views,mean_views,max_views,min_views]
        sum_mean_max_min_view.append(l0)
    return sum_mean_max_min_view

#对论文被浏览次数计算和(sum_view)、均值(mean_view)、最大值(max_view)、最小值(min_view)
views_lst = process_view_num(authors_id,author_papers_dict)
views_lst

[[21262, 708.7333333333333, 2008, 77],
 [104646, 1395.28, 4436, 294],
 [50628, 1875.111111111111, 8449, 153],
 [104357, 4537.260869565217, 24208, 703],
 [46400, 1718.5185185185185, 3912, 150],
 [45794, 2180.6666666666665, 5259, 336],
 [174760, 4064.186046511628, 9005, 639],
 [39357, 855.5869565217391, 3076, 77],
 [33246, 831.15, 3193, 77],
 [226309, 1870.3223140495868, 7739, 102],
 [333162, 3582.3870967741937, 17333, 413],
 [77045, 1328.3620689655172, 11303, 157],
 [47105, 1207.820512820513, 4533, 117],
 [92540, 1285.2777777777778, 5545, 117],
 [211691, 1545.1897810218977, 9509, 127],
 [78072, 667.2820512820513, 3793, 67],
 [122582, 1551.6708860759493, 5184, 144],
 [11589, 463.56, 2315, 112],
 [25020, 556.0, 1928, 112],
 [21808, 545.2, 1525, 112]]

### 对论文的发表日期进行分析

以上对每个作者发表论文的浏览次数进行了统计分析，但是我们还期望了解关于每个作者发表论文日期的更多信息，而每个作者的论文的发表日期一般不同，现在我们对每个作者发表论文的publishonline日期进行分析。

In [77]:

def get_max_publishonline(year_num_dict):
    """
    功能：
        计算论文数量最多的年份，当年份存在多个时，则保留最近的年份和对应的论文数量
    参数：
        year_num_dict：{年份：论文数量}
    返回值：
        以列表形式返回论文的数量最多的最近年份和对应的论文数量
    """
    # 论文的最多数量
    list6=[]
    list5=list(year_num_dict.values())
    max_year_num = max(list5)
    # 论文数量最多的年份
    num=list5.count(max_year_num)
    if num==1:
        max_year = list(year_num_dict.keys())[list(year_num_dict.values()).index(max_year_num)]
    # TODO
    # 计算论文的最多数量，以及对应的最近年份
    else:
        for item in enumerate(list5):
            if item[1]==max_year_num:
                list6.append(item[0])
        max_year=max([list(year_num_dict.keys())[i] for i in list6])         
    return [max_year, max_year_num]

In [72]:
def get_year_num_dict(year_pubonline):
    # 以字典形式返回作者每年发表的论文数量，year_num_dict = {年份：论文数量}
    year_num_only = np.unique(year_pubonline)
    year_num_dict = {}
    for year_num in year_num_only:
        year_num_dict[year_num] = year_pubonline.count(year_num)
    return year_num_dict

得到发表最多论文的年份，及该年发表的论文数量，以字典形式dict_year_num返回

In [75]:
def get_publishonline_year(author_id,author_papers_dict):  
    """
    功能：
        根据authorID，得到作者发表论文的年份
    参数：
        author_id：作者的authorID号
        author_papers_dict：包含了每个作者的paper数据，键为authorID，值为paper数据
    返回值：
        得到作者发表论文的年份列表
    """   
    # 年份列表
    items=author_papers_dict[author_id]
    year_pubonline = []
    for item in items:
        year_pubonline.append(item[10].split('-')[0])
    # TODO
    # 将发表日期进行数据转换，将得到的年份存储到年份列表中   
       
    return get_year_num_dict(year_pubonline)

依据作者列表，计算作者每年发表的论文数量，得到论文数量最多的年份，当年份存在多个时，则获取最近的年份，同时，获取该年份对应的论文次数

In [78]:
year_num_lst = []
for author_id in authors_id:
    year_num = get_publishonline_year(author_id,author_papers_dict)
    year_num_lst.append(get_max_publishonline(year_num))
# year_num_lst是存储了年份和论文数量的列表
# year_num_lst的每个元素是一个列表，这个列表为[publish_max_year,publish_year_num]，其中，第一个元素是authorID对应的论文数量最多的最近年份，第二个元素是论文的数量
print(year_num_lst)

[['2002', 4], ['2015', 8], ['2001', 4], ['1997', 4], ['1998', 5], ['2011', 5], ['2005', 6], ['2009', 5], ['2011', 4], ['2016', 9], ['2011', 9], ['2011', 7], ['2005', 4], ['2012', 6], ['2010', 10], ['2005', 11], ['2008', 8], ['1996', 6], ['2004', 6], ['1996', 6]]


### 合并数据，存入文件

In [88]:
file_path = './dataFile/author_paper_data.txt'
l7=[]
for i,j,k in zip(author_paper_data,views_lst,year_num_lst):
    l=i+j+k
    l7.append(l)
print(l7)
with open(file_path,'w') as fn:
    # TODO
    # 根据authors_id列表的authorID号，将得到的数据列表author_paper_data(author_id,papers_num)、
    # views_lst(sum_view,mean_view,max_view,min_view)、year_num_lst(publish_max_year,publish_year_num)进行合并，将合并后的数据进行输出
    # 并且，将合并后的数据存入txt文件
    for i in l7:
        fn.write(str(i)+'\n')    

[[5, 30, 21262, 708.7333333333333, 2008, 77, '2002', 4], [18, 75, 104646, 1395.28, 4436, 294, '2015', 8], [20, 27, 50628, 1875.111111111111, 8449, 153, '2001', 4], [22, 23, 104357, 4537.260869565217, 24208, 703, '1997', 4], [26, 27, 46400, 1718.5185185185185, 3912, 150, '1998', 5], [29, 21, 45794, 2180.6666666666665, 5259, 336, '2011', 5], [31, 43, 174760, 4064.186046511628, 9005, 639, '2005', 6], [48, 46, 39357, 855.5869565217391, 3076, 77, '2009', 5], [53, 40, 33246, 831.15, 3193, 77, '2011', 4], [57, 121, 226309, 1870.3223140495868, 7739, 102, '2016', 9], [64, 93, 333162, 3582.3870967741937, 17333, 413, '2011', 9], [65, 58, 77045, 1328.3620689655172, 11303, 157, '2011', 7], [69, 39, 47105, 1207.820512820513, 4533, 117, '2005', 4], [70, 72, 92540, 1285.2777777777778, 5545, 117, '2012', 6], [73, 137, 211691, 1545.1897810218977, 9509, 127, '2010', 10], [74, 117, 78072, 667.2820512820513, 3793, 67, '2005', 11], [79, 79, 122582, 1551.6708860759493, 5184, 144, '2008', 8], [89, 25, 11589, 

### 关闭数据库

In [89]:
conn.commit()
conn.close()